In [ ]:
import os
import pandas as pd
import rasterio
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats
from shapely.geometry import Point, box
from shapely.wkt import loads
from shapely.geometry import Point
import shapely as sp

pd.set_option('display.max_columns', None)

import warnings

warnings.filterwarnings("ignore")

## Get County Level Suitability Scores

In [5]:
suitability_block_group = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/regression/cleaned data bg/block_group_suitability_scores.csv", dtype={"GEOID": str})
FIPS = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/FIPS.csv", dtype={"FIPS State": str, "FIPS County": str})
suitability_block_group = suitability_block_group.merge(FIPS, left_on=["State", "County Name"], right_on=["State", "County Name"], how="left")
suitability_proj = pd.read_csv("data/suitability_data_project_level.csv", dtype={"GEOID": str, 'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'BLKGRPCE': str})

In [6]:
suitability_block_group

,GHI,Protected_Land,Habitat,Slope,Population_Density,Distance_to_Substation,Land_Cover,County Name,State,GEOID,TRACTCE,BLKGRPCE,FIPS State,FIPS County
0,20.000000,93.000000,92.000000,64.900000,10.000000,100.000000,67.600000,Russell,Alabama,011130303001,30300,1,01,113
1,90.000000,49.286809,68.606352,31.818498,99.692854,74.398245,86.935297,Mohave,Arizona,040159534062,953406,2,04,015
2,85.294525,94.656059,8.829808,61.149437,100.000000,50.000000,66.933254,Apache,Arizona,040019450021,945002,1,04,001
3,90.000000,99.765625,72.734375,99.687500,14.531250,50.000000,72.453125,Yuma,Arizona,040270003012,301,2,04,027
4,15.000000,100.000000,44.137931,23.922414,10.000000,50.000000,60.275862,Benton,Arkansas,050070208033,20803,3,05,007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237799,15.000000,79.285714,20.000000,100.000000,11.724138,50.000000,69.224138,Sussex,Delaware,100050513121,51312,1,10,005
237800,5.000000,100.000000,97.125000,100.000000,19.656250,71.875000,79.050000,Bay,Michigan,260172855005,285500,5,26,017
237801,5.000000,100.000000,97.000000,89.800000,10.000000,51.875000,67.250000,Oakland,Michigan,261251608005,160800,5,26,125
237802,5.000000,100.000000,96.315789,87.368421,10.000000,75.789474,71.736842,Hennepin,Minnesota,270530235023,23502,3,27,053


In [10]:
# List of factor to check
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']
NO_MATCHES = 10

dfs = {}
for factor in factors:
    # Sort and filter to get the factors suitability at county level
    factor_suitability = suitability_block_group.sort_values(factor, ascending=True)
    factor_suitability = factor_suitability[['State', 'County Name', 'GEOID'] + [factor]].dropna()
    
    suitability_factor_proj = suitability_proj[['GEOID', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE'] + [factor]]
    
    
    # Get the matches with teh projects
    project_matches = []

    for i in range(len(factor_suitability)):
        # get the state-county-fp
        state_county_fp = factor_suitability.iloc[i]["GEOID"]
        if state_county_fp in suitability_factor_proj["GEOID"].values:
            # add the rank of the project
            proj_matches = suitability_factor_proj[suitability_factor_proj["GEOID"] == state_county_fp]
            # Mean GHI of the projects
            proj_matches_mean_factor = proj_matches[factor].mean()
            # construct the row
            row = factor_suitability.iloc[i]
            row[f"Project_Mean_{factor}"] = proj_matches_mean_factor
            row["County Rank"] = i + 1
            project_matches.append(row)
            
        # Break when we get x matches
        if len(project_matches) >= NO_MATCHES:
            break
        
    dfs[factor] = pd.DataFrame(project_matches)

In [12]:
dfs['GHI']

,State,County Name,GEOID,GHI,Project_Mean_GHI,County Rank
199056,New York,Ulster,361119547001,5.0,255.0,122
198602,Michigan,Leelanau,260899706012,5.0,255.0,523
198652,Illinois,Marshall,171239614001,5.0,255.0,608
198996,Massachusetts,Hampshire,250158226072,5.0,255.0,692
82777,New Jersey,Middlesex,340230088012,5.0,255.0,710
82922,Minnesota,Le Sueur,270799504004,5.0,255.0,805
81182,Massachusetts,Bristol,250056332002,5.0,255.0,837
166622,New Jersey,Essex,340130209023,5.0,255.0,861
154511,Oregon,Yamhill,410710309002,5.0,15.0,875
199718,Minnesota,Carver,270190902004,5.0,255.0,959
